# Формирование решения
Данный ноутбук предназначен для обработки тестового набора данных обученной нейронной сетью и формирования итогового файла с результатами.

### Объявление вспомогательных функций

In [1]:
def crop_predict(markup, image, ROWS, COLS, h_step, w_step, variant, save):
    for i, rect in enumerate(crop_submission(ROWS, COLS, h_step, w_step, variant)):
        y_0, y_1, x_0, x_1 = rect
        cimage = copy.deepcopy(image[y_0:y_1, x_0:x_1])

        results = model(cimage, size=640, augment=False)

        # Обработка детекций
        for det in results.pandas().xyxy[0].iterrows():
            xmin, ymin, xmax, ymax, confidence, clsnum, name = det[1].values
            if confidence >= conf:
                # print(confidence)
                markup.append([xmin + x_0, ymin + y_0, xmax + x_0, ymax + y_0, confidence])
                cv2.rectangle(cimage,(int(xmin), int(ymin)),(int(xmax), int(ymax)),(0,255,0), 3)
        if save:
            cv2.imwrite(f'{yolo_experiment}images/{filename}_{i}_{variant}.jpg', cimage)

In [3]:
def transform(image, markup, cv_flip, y_flip, x_flip):
    
    image_tr = image.copy()
    markup_tr = []
    
    if y_flip or x_flip:
        image_tr = cv2.flip(image_tr, cv_flip)

    # Цикл разбиения картинки
    crop_predict(markup_tr, image_tr, ROWS, COLS, h_step, w_step, "B", save)

    if COLS >= 2:
        crop_predict(markup_tr, image_tr, ROWS, COLS-1, h_step, w_step, "C", save)

    if ROWS >= 2:
        crop_predict(markup_tr, image_tr, ROWS-1, COLS, h_step, w_step, "R", save)

    # image = cv2.flip(image, 0)
    if (y_flip or x_flip) and len(markup_tr) > 0:
        markup_tr = flip_bbox(np.array(markup_tr), (height, width), y_flip=y_flip, x_flip=x_flip)
        markup_tr = markup_tr.tolist()
        
    for mark in markup_tr:
        markup.append(mark)

    return markup

### Обработка изображений при помощи НС

In [20]:
import os
import torch
import cv2
from pathlib import Path

import sys
sys.path.append('/app/src')
from transforms import crop_submission
from nms import non_max_suppression_fast
from markup_convertors import flip_bbox

import numpy as np
import copy
import json
import pandas as pd
import tqdm
import glob

conf = 84
nms = 0.30
exp_name = f"_{conf}r_rot_area"
yolo_experiment = "/app/train_logs/yolov5x_dv7/"
test_images_path = "/app/data/raw/test_dataset_test/test/"
weights = "weights/best.pt"

Path(yolo_experiment + "images" + exp_name).mkdir(parents=True, exist_ok=True)

# Загрузка модели
model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_experiment + weights, force_reload=True)

conf = conf / 100

df = pd.DataFrame(columns=['ID_img', 'region_shape'])

for img_path in tqdm.tqdm(glob.glob(test_images_path + "*")):

    image = cv2.imread(img_path)
    filename = os.path.splitext(os.path.basename(img_path))[0]

    COLS = 4
    ROWS = 3

    height, width, ch = image.shape

    COLS = max(int(width / 1200),1)
    ROWS = max(int(height / 1000),1)

    w_step = width / COLS
    h_step = height / ROWS
    
    area = width * height

    save = False

    markup = []
    
    markup = transform(image, markup, 0, False, False)
    markup = transform(image, markup, 0, True, False)
    markup = transform(image, markup, 1, False, True)
    markup = transform(image, markup, -1, True, True)

    # NMS
    markup_nms = non_max_suppression_fast(np.array(markup), nms)

    if len(markup_nms) == 0:
        df = df.append({'ID_img':os.path.basename(img_path), 'region_shape': 0}, ignore_index=True)

    else:
        shapes = []
        for bbox in markup_nms:
            width = (bbox[3] - bbox[1]) / 2
            height = (bbox[2] - bbox[0]) / 2
            r = int(( width + height ) /2 ) * 2
            y = int(bbox[1] + width)
            x = int(bbox[0] + height)
            if r**2 * 100 / area < 0.18:
                cv2.circle(image, (x,y), radius=r, color=(0, 0, 255), thickness=3) # Отрисовка детекций на изображении
                cv2.putText(image, str(np.round(bbox[4],3)),(x,y), cv2.FONT_HERSHEY_PLAIN, 4, (255, 255, 255), 2, cv2.LINE_AA)
                shapes.append(str(json.dumps({"cx":x,"cy":y,"r":r})))
        
        df = df.append({'ID_img':os.path.basename(img_path), 'region_shape': sorted(shapes)}, ignore_index=True)

        cv2.imwrite(f'{yolo_experiment}images{exp_name}/{filename}_all.jpg', image)
        
df.to_csv(f'{yolo_experiment}submission_{yolo_experiment.split("/")[-2]}{exp_name}.csv', index=False, sep=",") 

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/denis/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-11-6 Python-3.8.10 torch-1.9.1+cu102 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5936MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 
100%|██████████| 3435/3435 [26:02<00:00,  2.20it/s]  
